# 🎓 Fine-tuning - 모델 파인튜닝

이 노트북은 beanllm의 파인튜닝 기능을 학습합니다. 커스텀 데이터셋으로 모델을 학습할 수 있습니다.

## 📋 목차

- [1. 파인튜닝 기본](#1-파인튜닝-기본)
- [2. 데이터셋 준비](#2-데이터셋-준비)
- [3. 학습 실행](#3-학습-실행)
- [4. 모델 평가](#4-모델-평가)

---

## 🎯 학습 목표

이 노트북을 완료하면:
- ✅ 파인튜닝의 개념을 이해할 수 있습니다
- ✅ 데이터셋을 준비할 수 있습니다
- ✅ 모델을 파인튜닝할 수 있습니다
- ✅ 파인튜닝된 모델을 평가할 수 있습니다

---

## 📚 사전 요구사항

- [01_setup_and_installation.ipynb](01_setup_and_installation.ipynb) 완료
- [02_core_client.ipynb](02_core_client.ipynb) 완료


## 1. 파인튜닝 기본

파인튜닝은 사전 학습된 모델을 특정 작업에 맞게 조정하는 과정입니다. 대규모 데이터로 사전 학습된 모델을 자신의 도메인이나 작업에 맞게 추가 학습시켜 성능을 향상시킬 수 있습니다.

> 💡 **파인튜닝의 핵심**:
> - **사전 학습된 모델 활용**: 이미 학습된 지식과 패턴 활용
> - **커스텀 데이터셋으로 학습**: 자신의 도메인 데이터로 특화
> - **특정 작업에 최적화**: 특정 작업의 성능 향상
> - **비용 효율성**: 처음부터 학습하는 것보다 훨씬 저렴

> 💡 **파인튜닝 vs 프롬프트 엔지니어링**:
> - **프롬프트 엔지니어링**: 빠르고 비용이 낮지만 제한적
> - **파인튜닝**: 더 정확하고 일관적이지만 시간과 비용 소요
> - **하이브리드**: 프롬프트 엔지니어링 + 파인튜닝 조합

> 💡 **파인튜닝이 유용한 경우**:
> - 특정 도메인 용어와 컨텍스트가 필요한 경우
> - 일관된 형식의 응답이 필요한 경우
> - 프롬프트만으로는 원하는 결과를 얻기 어려운 경우
> - 대량의 도메인 특화 데이터가 있는 경우


In [ ]:
from beanllm import FineTuner
import json

# ============================================
# 파인튜닝 기본
# ============================================

print("=" * 60)
print("파인튜닝 기본")
print("=" * 60)

# 파인튜닝 준비
print("\n[FineTuner 생성]")
# fine_tuner = FineTuner(
#     base_model="gpt-4o",  # 또는 "gpt-3.5-turbo"
#     provider="openai"
# )

print("✅ FineTuner 준비 완료!")
print("\n💡 파인튜닝 프로세스:")
print("  1. 데이터셋 준비 (JSONL 형식)")
print("  2. 데이터 검증 및 업로드")
print("  3. 학습 작업 시작")
print("  4. 학습 진행 상황 모니터링")
print("  5. 모델 평가 및 테스트")
print("  6. 모델 배포 및 사용")

print("\n💡 지원하는 Provider:")
print("  - OpenAI Fine-tuning API: 클라우드 기반, 간편함")
print("  - Axolotl: 로컬 학습, 완전한 제어")

# ============================================
# 파인튜닝 옵션 설명
# ============================================

print("\n" + "=" * 60)
print("파인튜닝 옵션")
print("=" * 60)

fine_tuning_options = {
    "base_model": {
        "description": "기본 모델 선택",
        "options": ["gpt-4o", "gpt-3.5-turbo"],
        "recommendation": "작업 복잡도에 따라 선택",
    },
    "training_epochs": {
        "description": "학습 에포크 수",
        "options": "1-10 (일반적으로 3-5)",
        "recommendation": "데이터셋 크기에 따라 조정",
    },
    "learning_rate": {
        "description": "학습률",
        "options": "자동 또는 수동 설정",
        "recommendation": "일반적으로 자동 설정 권장",
    },
    "batch_size": {
        "description": "배치 크기",
        "options": "자동 또는 수동 설정",
        "recommendation": "데이터셋 크기에 따라 자동 조정",
    },
}

print("\n📊 파인튜닝 주요 옵션:")
for option, info in fine_tuning_options.items():
    print(f"\n[{option}]")
    print(f"  설명: {info['description']}")
    print(f"  옵션: {info['options']}")
    print(f"  권장: {info['recommendation']}")

print("\n💡 파인튜닝 팁:")
print("  - 데이터셋 품질이 가장 중요합니다")
print("  - 최소 10개, 권장 50-100개 이상의 샘플")
print("  - 다양하고 대표적인 예제 포함")
print("  - 검증 데이터셋도 준비하는 것이 좋습니다")

print("\n" + "=" * 60)
print("✅ 파인튜닝 기본 학습 완료!")
print("=" * 60)

## 2. 데이터셋 준비

파인튜닝을 위한 데이터셋을 준비합니다. 데이터셋의 품질이 파인튜닝 성공의 핵심입니다.

> 💡 **데이터셋 요구사항**:
> - **JSONL 형식**: 각 줄이 하나의 JSON 객체
> - **메시지 형식**: OpenAI 형식 (system, user, assistant)
> - **최소 샘플 수**: 10개 이상 (권장: 50-100개 이상)
> - **품질**: 다양하고 대표적인 예제

> 💡 **데이터셋 구조**:
> ```json
> {
>   "messages": [
>     {"role": "system", "content": "시스템 메시지"},
>     {"role": "user", "content": "사용자 질문"},
>     {"role": "assistant", "content": "어시스턴트 응답"}
>   ]
> }
> ```

> ⚠️ **주의사항**:
> - 각 메시지는 role과 content 필드를 가져야 합니다
> - system 메시지는 선택적이지만 권장됩니다
> - assistant 응답은 명확하고 일관되어야 합니다


In [ ]:
# ============================================
# 데이터셋 준비
# ============================================

print("=" * 60)
print("데이터셋 준비")
print("=" * 60)

# 데이터셋 예제 (JSONL 형식)
dataset_examples = [
    {
        "messages": [
            {"role": "system", "content": "당신은 도움이 되는 AI 어시스턴트입니다."},
            {"role": "user", "content": "안녕하세요"},
            {"role": "assistant", "content": "안녕하세요! 무엇을 도와드릴까요?"},
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "당신은 프로그래밍 전문가입니다."},
            {"role": "user", "content": "Python이란?"},
            {
                "role": "assistant",
                "content": "Python은 1991년에 발표된 고급 프로그래밍 언어입니다. 간결하고 읽기 쉬운 문법이 특징입니다.",
            },
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "당신은 데이터 과학 전문가입니다."},
            {"role": "user", "content": "머신러닝의 기본 개념을 설명해주세요."},
            {
                "role": "assistant",
                "content": "머신러닝은 데이터로부터 패턴을 학습하여 예측이나 분류를 수행하는 AI 기술입니다. 지도 학습, 비지도 학습, 강화 학습으로 분류됩니다.",
            },
        ]
    },
]

print("\n📊 데이터셋 예제 (JSONL 형식):")
for i, example in enumerate(dataset_examples, 1):
    print(f"\n[예제 {i}]")
    json_str = json.dumps(example, ensure_ascii=False, indent=2)
    print(json_str)

# JSONL 파일 생성 예제
print("\n[JSONL 파일 생성 예제]")
# import json
#
# dataset_file = "fine_tuning_dataset.jsonl"
#
# with open(dataset_file, "w", encoding="utf-8") as f:
#     for example in dataset_examples:
#         f.write(json.dumps(example, ensure_ascii=False) + "\n")
#
# print(f"✅ 데이터셋 파일 생성: {dataset_file}")

print("\n💡 데이터셋 요구사항:")
print("  - JSONL 형식: 각 줄이 하나의 JSON 객체")
print("  - messages 배열: system, user, assistant 메시지")
print("  - 최소 샘플 수: 10개 이상 (권장: 50-100개 이상)")
print("  - 다양성: 다양한 시나리오와 예제 포함")
print("  - 품질: 명확하고 일관된 응답")

# ============================================
# 데이터셋 검증
# ============================================

print("\n" + "=" * 60)
print("데이터셋 검증")
print("=" * 60)


def validate_dataset(dataset):
    """
    데이터셋 검증 함수

    데이터셋이 올바른 형식인지 확인합니다.
    """
    errors = []
    warnings = []

    if not dataset:
        errors.append("데이터셋이 비어있습니다")
        return errors, warnings

    # 최소 샘플 수 확인
    if len(dataset) < 10:
        warnings.append(f"샘플 수가 적습니다 ({len(dataset)}개). 최소 10개 이상 권장")

    # 각 샘플 검증
    for i, sample in enumerate(dataset):
        if "messages" not in sample:
            errors.append(f"샘플 {i+1}: 'messages' 키가 없습니다")
            continue

        messages = sample["messages"]
        if not isinstance(messages, list):
            errors.append(f"샘플 {i+1}: 'messages'가 리스트가 아닙니다")
            continue

        if len(messages) < 2:
            errors.append(f"샘플 {i+1}: 최소 2개의 메시지가 필요합니다 (user, assistant)")
            continue

        # 메시지 형식 검증
        has_user = False
        has_assistant = False

        for msg in messages:
            if "role" not in msg or "content" not in msg:
                errors.append(f"샘플 {i+1}: 메시지에 'role' 또는 'content'가 없습니다")
                continue

            if msg["role"] == "user":
                has_user = True
            elif msg["role"] == "assistant":
                has_assistant = True

        if not has_user:
            errors.append(f"샘플 {i+1}: 'user' 메시지가 없습니다")
        if not has_assistant:
            errors.append(f"샘플 {i+1}: 'assistant' 메시지가 없습니다")

    return errors, warnings


# 데이터셋 검증 실행
print("\n[데이터셋 검증 실행]")
errors, warnings = validate_dataset(dataset_examples)

if errors:
    print("\n❌ 오류:")
    for error in errors:
        print(f"  - {error}")
else:
    print("  ✅ 오류 없음")

if warnings:
    print("\n⚠️ 경고:")
    for warning in warnings:
        print(f"  - {warning}")
else:
    print("  ✅ 경고 없음")

print("\n💡 데이터셋 검증 팁:")
print("  - 파인튜닝 전에 반드시 검증")
print("  - 형식 오류는 학습 실패로 이어질 수 있음")
print("  - 샘플 수와 다양성 확인")
print("  - 응답 품질 확인")

print("\n" + "=" * 60)
print("✅ 데이터셋 준비 학습 완료!")
print("=" * 60)

In [ ]:
# ============================================
# 학습 실행
# ============================================

print("=" * 60)
print("학습 실행")
print("=" * 60)


async def start_fine_tuning():
    """
    파인튜닝 학습 시작 예제

    데이터셋을 업로드하고 학습을 시작합니다.
    """
    from beanllm import FineTuner

    # FineTuner 생성
    fine_tuner = FineTuner(base_model="gpt-3.5-turbo", provider="openai")  # 파인튜닝 가능한 모델

    # 1. 데이터셋 업로드
    print("\n[1단계] 데이터셋 업로드")
    dataset_file = "fine_tuning_dataset.jsonl"
    # file_id = await fine_tuner.upload_dataset(dataset_file)
    # print(f"  ✅ 데이터셋 업로드 완료: {file_id}")

    # 2. 학습 작업 시작
    print("\n[2단계] 학습 작업 시작")
    # job = await fine_tuner.start_training(
    #     training_file=file_id,
    #     validation_file=None,  # 선택적: 검증 데이터셋
    #     hyperparameters={
    #         "n_epochs": 3,  # 학습 에포크 수
    #     }
    # )
    #
    # print(f"  ✅ 학습 작업 시작: {job.id}")
    # print(f"  📊 상태: {job.status}")

    # 3. 학습 진행 상황 모니터링
    print("\n[3단계] 학습 진행 상황 모니터링")
    # while job.status not in ["succeeded", "failed", "cancelled"]:
    #     job = await fine_tuner.get_job_status(job.id)
    #     print(f"  진행 중... 상태: {job.status}")
    #     if hasattr(job, 'progress'):
    #         print(f"  진행률: {job.progress}%")
    #     await asyncio.sleep(10)  # 10초마다 확인

    # 4. 학습 완료
    # if job.status == "succeeded":
    #     print(f"\n✅ 학습 완료!")
    #     print(f"  모델 ID: {job.fine_tuned_model}")
    #     print(f"  학습 시간: {job.training_duration}초")
    #     return job.fine_tuned_model
    # else:
    #     print(f"\n❌ 학습 실패: {job.status}")
    #     return None

    print("\n💡 학습 실행 팁:")
    print("  - 학습은 비동기로 진행됩니다")
    print("  - 진행 상황을 주기적으로 확인")
    print("  - 학습 완료까지 시간이 걸릴 수 있음")
    print("  - 비용을 미리 확인하세요")


# 실행 (주석 해제)
# model_id = await start_fine_tuning()

# ============================================
# 파인튜닝된 모델 사용
# ============================================

print("\n" + "=" * 60)
print("파인튜닝된 모델 사용")
print("=" * 60)


async def use_fine_tuned_model():
    """
    파인튜닝된 모델 사용 예제

    학습이 완료된 모델을 사용합니다.
    """
    from beanllm import Client

    # 파인튜닝된 모델 ID (학습 완료 후 받은 ID)
    fine_tuned_model_id = "ft:gpt-3.5-turbo:your-org:custom-model:abc123"

    # 파인튜닝된 모델로 Client 생성
    client = Client(model=fine_tuned_model_id)

    # 모델 사용
    # response = await client.chat(
    #     messages=[
    #         {"role": "user", "content": "Python이란?"}
    #     ]
    # )
    #
    # print(f"✅ 파인튜닝된 모델 응답:")
    # print(f"  {response.content}")

    print("\n💡 파인튜닝된 모델 사용:")
    print("  - 일반 모델과 동일하게 사용 가능")
    print("  - 파인튜닝된 특성 반영")
    print("  - 더 정확하고 일관된 응답")
    print("  - 도메인 특화 응답 생성")


# 실행 (주석 해제)
# await use_fine_tuned_model()

print("\n" + "=" * 60)
print("✅ 학습 실행 학습 완료!")
print("=" * 60)

In [ ]:
# ============================================
# 모델 평가
# ============================================

print("=" * 60)
print("모델 평가")
print("=" * 60)


async def evaluate_fine_tuned_model():
    """
    파인튜닝된 모델 평가 예제

    학습된 모델의 성능을 평가합니다.
    """
    from beanllm import Client

    # 원본 모델과 파인튜닝된 모델
    original_model = Client(model="gpt-3.5-turbo")
    fine_tuned_model = Client(model="ft:gpt-3.5-turbo:your-org:custom-model:abc123")

    # 테스트 케이스
    test_cases = ["Python이란?", "머신러닝의 기본 개념은?", "데이터 과학이란?"]

    print("\n📊 테스트 케이스:")
    for i, case in enumerate(test_cases, 1):
        print(f"  {i}. {case}")

    # 각 모델로 테스트
    print("\n[모델 비교 평가]")
    # for case in test_cases:
    #     print(f"\n  질문: {case}")
    #
    #     # 원본 모델 응답
    #     original_response = await original_model.chat(
    #         messages=[{"role": "user", "content": case}]
    #     )
    #     print(f"  원본 모델: {original_response.content[:100]}...")
    #
    #     # 파인튜닝된 모델 응답
    #     fine_tuned_response = await fine_tuned_model.chat(
    #         messages=[{"role": "user", "content": case}]
    #     )
    #     print(f"  파인튜닝 모델: {fine_tuned_response.content[:100]}...")

    print("\n💡 모델 평가 팁:")
    print("  - 다양한 테스트 케이스로 평가")
    print("  - 원본 모델과 비교")
    print("  - 도메인 특화 성능 확인")
    print("  - 실제 사용 사례로 테스트")


# 실행 (주석 해제)
# await evaluate_fine_tuned_model()

print("\n" + "=" * 60)
print("✅ 모델 평가 학습 완료!")
print("=" * 60)

## 3. 학습 실행

데이터셋이 준비되면 파인튜닝 학습을 시작할 수 있습니다.

> 💡 **학습 프로세스**:
> - 데이터셋 업로드: Provider에 데이터셋 업로드
> - 학습 작업 시작: 파인튜닝 작업 생성
> - 진행 상황 모니터링: 학습 진행 상황 추적
> - 완료 대기: 학습 완료까지 대기 (수 분 ~ 수 시간)
> - 모델 ID 획득: 완료된 모델의 ID 저장

> ⚠️ **주의사항**:
> - 학습은 시간이 걸릴 수 있습니다 (데이터셋 크기에 따라)
> - 비용이 발생할 수 있습니다 (Provider별로 다름)
> - 학습 중에는 모델을 사용할 수 없습니다

## 4. 모델 평가

파인튜닝된 모델의 성능을 평가합니다.

> 💡 **평가 방법**:
> - **정성적 평가**: 샘플 입력으로 응답 품질 확인
> - **정량적 평가**: 테스트 데이터셋으로 정확도 측정
> - **비교 평가**: 원본 모델과 성능 비교
> - **도메인 특화 평가**: 특정 도메인에서의 성능 평가

Fine-tuning을 마스터했습니다! 다음 튜토리얼로 진행하세요:

### 📚 추천 학습 순서

1. **[14_production_features.ipynb](14_production_features.ipynb)** - Production Features
   - 프로덕션 환경 최적화
   - 모니터링 및 로깅

2. **[15_distributed_architecture.ipynb](15_distributed_architecture.ipynb)** - Distributed Architecture
   - 분산 시스템 구성
   - Redis & Kafka 활용

### 🔗 관련 문서

- [API Reference](../API_REFERENCE.md#fine-tuning) - Fine-tuning API 상세 문서
- [README.md](../../README.md) - 프로젝트 개요

---

**✅ Fine-tuning 완료! 다음 노트북으로 진행하세요.**
